In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
# URL alvo
url = 'https://jucemg.mg.gov.br/pagina/140/leiloeiros-ordem-alfabetica'

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [5]:
# Enviar requisição GET para o site
response = requests.get(url, headers=headers)

In [6]:
response.status_code

200

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [8]:
# Encontrar o conteúdo dentro da tag 'article'
html = soup.find('article')

In [9]:
print(html)

<article>
<h1 class="title title--md title--dark">A a Z</h1>
<div class="separator"></div>
<div class="row">
<div class="col-md-6 pull-left text-left">
</div>
<div class="col-md-6 pull-right text-right">
</div>
</div>
<section class="section__xs">
<div style="clear: both">
<p><strong>Adriana Pires Amancio</strong><br/>
Matrícula: 1062 de 30/05/2017<br/>
Preposto: Não possui preposto.<br/>
Caixa Postal 8098 - Belo Horizonte - MG - CEP 31.310-970<br/>
Telefones: (31) 3564-1314 / (31) 99966-3086<br/>
<a href="mailto:adriana.pires@apaleiloes.com.br">adriana.pires@apaleiloes.com.br</a><br/>
<a href="http://www.apaleiloes.com.br">www.apaleiloes.com.br</a> <br/>
<br/>
<strong>Adriano Apolinário Leão de Oliveira</strong><br/>
Matrícula: 820 de 06/08/2010<br/>
Preposto: Não possui preposto.<br/>
Iguatama - MG<br/>
Telefones: (37) 3353-2301 / (37) 99915-8851<br/>
<a href="mailto:contato@adrianoleiloeiro.com.br">contato@adrianoleiloeiro.com.br</a></p>
<p><strong>Aécio Reis Pedrosa</strong><br/>
M

In [10]:
# Lista de siglas de estados
lista_siglas = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 
                'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 
                'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 
                'SP', 'TO']

In [117]:
# Inicializando listas para armazenar os dados
nomes = []
matriculas = []
estados = []
telefones = []
emails = []

for leiloeiro in html.find_all('p'):
    # Separar leiloeiros se houver mais de um (identificado por uma nova linha ou outro critério)
    leiloeiros_info = leiloeiro.text.split('\n\n')  # Separar por quebras de linha

    for info in leiloeiros_info:
       
        # Extrair o nome do leiloeiro
        nome = info.split('\n')[0].strip()
        nomes.append(nome)

        # Extrair a matrícula
        matricula_info = info.find("Matrícula:")
        if matricula_info != -1:
            matricula = info[matricula_info:].split(':')[1].strip().split('\r')[0]
            matriculas.append(matricula)
        else:
            matriculas.append(None)

        # Extrair o estado
        endereco_info = info.find("Prepost")
        if endereco_info != -1:
            endereco = info[endereco_info:].split('\n')[1].split('\r')[0]
        


        def extrair_sigla(endereco):
            for sigla in lista_siglas:
            # Procurar se a sigla está presente após um espaço ou hífen
                if re.search(rf"[\s\-]{sigla}(?:[\s\-]|$)", endereco):
                    return sigla
            return None

        estados.append(extrair_sigla(endereco))


        # Extrair os telefones
        telefone_info = info.find("Telefone")
        if telefone_info != -1:
            telefone = info[telefone_info:].split(':')[-1].split('\n')[0].strip().replace(' / ', ', ')
            telefones.append(telefone)
        else:
            telefones.append(None)
        
        # Extrair os e-mails
        
        email_links = re.findall(r'[\w\.-]+@[\w\.-]+', info)
        email = ', '.join(email_links) if email_links else None
        emails.append(email)


In [118]:
# Criar um DataFrame com os dados extraídos
df = pd.DataFrame({
    'Nome': nomes,
    'Matrícula': matriculas,
    'Estado': estados,
    'telefone' : telefones,
    'Email' : emails
})
# Exibir o DataFrame
df.head(200)

,Nome,Matrícula,Estado,telefone,Email
0,Adriana Pires Amancio,1062 de 30/05/2017,MG,"(31) 3564-1314, (31) 99966-3086",adriana.pires@apaleiloes.com.br
1,Adriano Apolinário Leão de Oliveira,820 de 06/08/2010,MG,"(37) 3353-2301, (37) 99915-8851",contato@adrianoleiloeiro.com.br
2,Aécio Reis Pedrosa,1283 de 19/04/2022,MG,(31) 99535-8075,aeciopedrosaleiloeiro@gmail.com
3,Alessandro de Assis Teixeira,992 de 06/04/2015,MG,(35) 99228-1011,"contato@alessandroteixeiraleiloes.com.br, ales..."
4,Alex Willian Hoppe (MATRÍCULA SUPLEMENTAR),1199 de 04/12/2020,SC,(47) 3622-5164,contato@hoppeleiloes.com.br
...,...,...,...,...,...
179,Viviane Garzon Corrêa,387 de 22/02/2000,MG,"(31) 3292-8463, (31) 3048-4300",viviane@bolsadeleiloes.com.br
180,Wellington de Matos Silva,1162 de 24/01/2020,MG,(31) 99728-3092,wsleiloes@yahoo.com
181,Wendell Marcel Calixto Felix (MATRÍCULA SUPLEM...,1306 de 11/08/2022,SP,(11) 99837-4277,"wendell.calixto@hotmail.com, wmcfelix@gmail.com"
182,Wesley Oliveira Ascanio (MATRÍCULA SUPLEMENTAR),1399 de 01/07/2024,SP,(11) 98771-4373,wesley@tabaleiloes.com.br


In [119]:
df.to_excel(r'C:\Users\gabriel\OneDrive\Documentos\Freelancers\Leiloeiros_nas_Juntas_Comerciais_do_Brasil.xlsx', index=False)
